In [11]:
import os
import pandas as pd
import spacy
from tqdm import tqdm_notebook as tqdm

# Make word clouds of context

## Get context

In [3]:
seed_path = '/home/michael/school/11-830/hw1/seed_words.txt'
with open(seed_path) as f:
    seed = f.read().splitlines()
    
seed

['agender',
 'androgyny',
 'androgynyous',
 'bisexual',
 'bigender',
 'butch',
 'cisgender',
 'gay',
 'gays',
 'gender-fluid',
 'genderqueer',
 'lesbian',
 'lesbians',
 'LGBT',
 'LGBTQ',
 'queer',
 'transgender',
 'same-sex']

In [6]:
# Load data
lgbtq_data = pd.read_csv('/home/michael/school/11-830/hw1/lgbt_news_corpus.csv', encoding='ISO-8859-1')
lgbtq_data

/home/michael/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (7,8,10,14,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Text,Title,Id,Count,Date,Category,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,Re ÛÏObama Alters Hospital Rules for Gay Righ...,Hospital Visiting Rights for Same-Sex Partners,54457293798810347c1a843e,1,2010-04-26T00:00:00Z,Opinion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,You cannot have people who have no expertise i...,"For Police Union Head, Review Board Proposal I...",4fd1d13d8eb7c8105d70bb37,3,1992-06-28T00:00:00Z,New York and Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,In an interview with Tracy Clark-Flory on Salo...,Why Asexuals DonÛªt Want to Be Invisible Anymore,54232178798810604c505435,4,2014-09-24T15:44:50Z,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,After we filed back out into the Tennessee sun...,"Dollywood: A Little Bit Country, a Little Bit Gay",53f782e67988105a42594f8f,28,2014-08-24T00:00:00Z,Travel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,''Gays and lesbians are a very good group of p...,San Francisco Toasts Gay Weddings,4fd24bb98eb7c8105d7eacb6,20,2004-02-29T00:00:00Z,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,The concept was brought to the division by Mae...,Time Inc. Considers Starting a Magazine for Ga...,4fd1e0ee8eb7c8105d727f1f,1,1994-01-24T00:00:00Z,Business,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Mr. Winslow urged any justice with qualms abou...,"Obey Same-Sex Marriage Law, Officials Told",4fd23c0b8eb7c8105d7d0ed5,2,2004-04-26T00:00:00Z,U.S.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"To the contrary, Mr. Lieberman argued, the law...","To Commit a Hate Crime, Must the Criminal Trul...",548b8e1d79881004157bba6d,18,2007-06-20T00:00:00Z,N.Y. / Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,All this affirmation of existing hostility mea...,"Don't Ask, Don't Tell, Don't Think",5005d0c21c22dfde670b0b49,2,2012-07-17T16:50:14Z,Opinion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,The fears by gay priests and seminarians inten...,Gay Men Ponder Impact Of Proposal by Vatican,4fd25f7e8eb7c8105d80f225,4,2005-09-23T00:00:00Z,U.S.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
len(lgbtq_data)

34620

In [8]:
lgbtq_data['Text']

0        Re ÛÏObama Alters Hospital Rules for Gay Righ...
1        You cannot have people who have no expertise i...
2        In an interview with Tracy Clark-Flory on Salo...
3        After we filed back out into the Tennessee sun...
4        ''Gays and lesbians are a very good group of p...
5        The concept was brought to the division by Mae...
6        Mr. Winslow urged any justice with qualms abou...
7        To the contrary, Mr. Lieberman argued, the law...
8        All this affirmation of existing hostility mea...
9        The fears by gay priests and seminarians inten...
10       Financial experts agree that the most importan...
11       The repeal of the out-of-state marriage ban wo...
12       Last year, for example, Secretary of State Hil...
13       She drew another ovation, too, with her respon...
14       But many gay people in their teens, 20s and 30...
15       Stuart Elliott, The TimesÛªs advertising colu...
16       While same-sex marriages are forging ahead in .

## Lowercase, tokenize

In [13]:
nlp = spacy.load('en', disable=['ner'])
docs = [nlp(doc) for doc in tqdm(lgbtq_data['Text'].tolist())]

Widget Javascript not detected.  It may not be installed or enabled properly.


15909/|/ 46%|| 15909/34620 [06:23<07:31, 41.47it/s]

KeyboardInterrupt: 

16875/|/ 49%|| 16875/34620 [07:03<07:25, 39.83it/s]

In [ ]:
lgbtq_data['toks'] = lgbtq_data['Text']